# NOTE
This notebook was created for the sole purpose of generating maps for the project website; the code was run locally and might be slightly deprecated due to small changes between the datasets used here and the finalized versions linked to from Google Drive

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import glob
import re

from math import radians, sin, cos, acos, log, pi, tan, asin,sqrt
from decimal import Decimal
from bokeh.plotting import figure, show, output_notebook
from bokeh.tile_providers import CARTODBPOSITRON,STAMEN_TERRAIN
from ast import literal_eval
from scipy import stats
from math import radians, sin, cos, acos, log, pi, tan, asin,sqrt
from decimal import Decimal
from bokeh.embed import components
from bokeh.resources import CDN
from bokeh.transform import factor_cmap
from bokeh.palettes import Category20b_20
from bokeh.models import ColumnDataSource, HoverTool

KeyboardInterrupt: 

In [ ]:
def merc(Coords):
    Coordinates = (Coords)
    lat = Coordinates[0]
    lon = Coordinates[1]
    
    r_major = 6378137.000
    x = r_major * radians(lon)
    scale = x/lon
    y = 180.0/pi * log(tan(pi/4.0 + 
        lat * (pi/180.0)/2.0)) * scale
    return (x, y)

In [ ]:
#Download from: https://drive.google.com/open?id=1AOv3CrNOysm8FOfgSLL8aWhcWGPRXgPb
airbnb_cr = pd.read_csv('/home/novais/Downloads/airbnb_cr.csv')

In [ ]:
airbnb_cr.crimes_5km_count=airbnb_cr.crimes_1km_count+airbnb_cr.crimes_2km_count+airbnb_cr.crimes_5km_count
airbnb_cr.crimes_2km_count=airbnb_cr.crimes_1km_count+airbnb_cr.crimes_2km_count
airbnb_cr.head()

In [ ]:
ids=list(airbnb_cr['listing_id'].unique())

In [ ]:
loc_sum=airbnb_cr.groupby(['loc_id']).sum()
loc_sum=loc_sum.reset_index()
loc_sum=loc_sum[['loc_id','crimes_1km_count','crimes_2km_count','crimes_5km_count']]
loc_sum=loc_sum.rename(index=str, columns={"crimes_1km_count": "crimes_1km_sum","crimes_2km_count": "crimes_2km_sum","crimes_5km_count": "crimes_5km_sum"})
loc_sum.head()

In [ ]:
loc_mean=airbnb_cr.groupby(['loc_id']).mean()
loc_mean=loc_mean.reset_index()
loc_mean=loc_mean[['loc_id','crimes_1km_count','crimes_2km_count','crimes_5km_count']]
loc_mean=loc_mean.rename(index=str, columns={"crimes_1km_count": "crimes_1km_mean","crimes_2km_count": "crimes_2km_mean","crimes_5km_count": "crimes_5km_mean"})
loc_mean.head()

In [ ]:
loc_count=loc_mean.merge(loc_sum,on='loc_id')
loc_count.head()

In [ ]:
airbnb_cr2=airbnb_cr.merge(loc_count,on='loc_id')
airbnb_cr2=airbnb_cr2.groupby('loc_id').mean()
airbnb_cr2=airbnb_cr2.reset_index()
airbnb_cr2=airbnb_cr2[['lat','lon','loc_id','crimes_1km_mean','crimes_2km_mean','crimes_5km_mean','crimes_1km_sum','crimes_2km_sum','crimes_5km_sum']]
airbnb_cr2['loc']=list(zip(airbnb_cr2.lat,airbnb_cr2.lon))
airbnb_cr2.head()

In [ ]:
# listings2['loc'] = listings2['loc'].apply(lambda x: literal_eval(x))
airbnb_cr2['coords_x'] = airbnb_cr2['loc'].apply(lambda x: merc(x)[0])
airbnb_cr2['coords_y'] = airbnb_cr2['loc'].apply(lambda x: merc(x)[1])

p1 = figure(title = "Listings and Crimes")
p1.xaxis.axis_label = 'Latitude'
p1.yaxis.axis_label = 'Longitude'

a1 = airbnb_cr2[['crimes_1km_sum','crimes_1km_mean','coords_x','coords_y']]
a2 = airbnb_cr2[['crimes_2km_sum','crimes_2km_mean','coords_x','coords_y']]
a5 = airbnb_cr2[['crimes_5km_sum','crimes_5km_mean','coords_x','coords_y']]

source5 = ColumnDataSource(data=dict(
    s=a5['crimes_5km_sum'],
    cx=a5['coords_x'],
    cy=a5['coords_y'],
    sizes = [50]*len(a5),
    names = ['5 km']*len(a5),
    colors = ['#F39A05']*len(a5)
    ))
source2 = ColumnDataSource(data=dict(
    s=a2['crimes_2km_sum'],
    cx=a2['coords_x'],
    cy=a2['coords_y'],
    sizes = [30]*len(a2),
    names = ['2 km']*len(a2),
    colors = ['#D34B08']*len(a2)
    ))
source1 = ColumnDataSource(data=dict(
    s=a1['crimes_1km_sum'],
    cx=a1['coords_x'],
    cy=a1['coords_y'],
    sizes = [10]*len(a1),
    names = ['1 km']*len(a1),
    colors = ['#B70B0D']*len(a1)
    ))



p1 = figure(x_axis_type="mercator", y_axis_type="mercator")
p1.add_tile(CARTODBPOSITRON)

p1.circle(x = 'cx',y = 'cy',source=source5,size='sizes',legend ='names',color='colors',fill_alpha=0.5)
p1.circle(x = 'cx',y = 'cy',source=source2,size='sizes',legend ='names',color='colors',fill_alpha=0.6)
p1.circle(x = 'cx',y = 'cy',source=source1,size='sizes',legend ='names',color='colors',fill_alpha=0.7)



p1.add_tools(HoverTool(
    tooltips=[
        ( 'Nº of occurrences', '@s')
    ],
    
    formatters={
        'Nº of occurrences' : 'numeral',   
    },

))

p1.legend.location = "top_right"
p1.legend.click_policy="hide"

p1_script, p1_div = components(p1)

In [ ]:
output_notebook()


In [ ]:
show(p1)

In [ ]:
print(p1_script)

In [ ]:
print(p1_div)